# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
%load_ext autoreload
%autoreload 2
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import torch
import matplotlib.pyplot as plt
%matplotlib inline
from p3_mad4pg import MAD4PG

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.10000000149011612
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
def train(check_pt_path,n_episodes=10000,isSolve=True):
    """Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    brain_name = env.brain_names[0]
    brain = env.brains[brain_name]
    
    avgscores=[]
    scores=[]
    scores1 = []
    scores2 = []# list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states= env_info.vector_observations
        score1 = 0
        score2 = 0
        while True:
            actions = multiagent.acts(states)
            env_info=env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            multiagent.step(states,actions, rewards, next_states, dones)
            states = next_states
            score1 += rewards[0]
            score2 += rewards[1]
            if dones[0]:
                break

        score=max(score1,score2)        
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        scores1.append(score1)
        scores2.append(score2)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="."*(i_episode%10))
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            avgscores.append(np.mean(scores_window))
        if np.mean(scores_window)>0.5 and isSolve:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(multiagent.mad4pg_agent[0].actor_local.state_dict(),check_pt_path+'_actor1_solved.pth')
            torch.save(multiagent.mad4pg_agent[1].actor_local.state_dict(),check_pt_path+'_actor2_solved.pth')
            isSolve=False
    torch.save(multiagent.mad4pg_agent[0].actor_local.state_dict(),check_pt_path+'_actor1_last.pth')
    torch.save(multiagent.mad4pg_agent[1].actor_local.state_dict(),check_pt_path+'_actor2_last.pth')
    return scores,scores1,scores2


In [ ]:
# train multi agent with D4PG
multiagent=MAD4PG(24,2,32,buffer_size = int(1e6),batch_size = 64,gamma = 0.99,tau = 1e-2,update_every = 6,num_mc_steps=7)
scores,scores1,scores2 = train(check_pt_path='multiagents_d4pg',n_episodes=20000)

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores1)), scores1)
plt.plot(np.arange(len(scores2)), scores2)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend(['agent1', 'agent2'], loc='lower right')
plt.show()



Episode 100	Average Score: 0.01........
Episode 200	Average Score: 0.00.........
Episode 300	Average Score: 0.00.........
Episode 400	Average Score: 0.00.........
Episode 500	Average Score: 0.00.........
Episode 532	Average Score: 0.00.........

In [ ]:
#Watch a mad4pg trained Agent
multiagent=MAD4PG(24,2,32,BUFFER_SIZE = int(1e6),BATCH_SIZE = 64,GAMMA = 0.99,TAU = 1e-2,LR_ACTOR = 1e-4,LR_CRITIC = 1e-4,WEIGHT_DECAY = 0,UPDATE_EVERY = 6,N_step=7)
multiagent.mad4pg_agent[0].actor_local.load_state_dict(torch.load('multiagents_d4pg_actor1_last.pth'))
multiagent.mad4pg_agent[1].actor_local.load_state_dict(torch.load('multiagents_d4pg_actor2_last.pth'))

env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86_64")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

for i in range(1, 4):                                      # play game for 3 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(2)                          # initialize the score (for each agent)
    while True:
        actions = multiagent.acts(states) # select an action (for each agent)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))
    


In [4]:
import numpy as np
import torch

In [5]:
x= np.random.randn(10)

In [6]:
y = torch.from_numpy(x).float().to('cpu')

In [12]:
y.unsqueeze(-2).shape

torch.Size([1, 10])

In [13]:
x=[1,2,3,4]

In [20]:
torch.tensor(sum(x)).unsqueeze(0).shape

torch.Size([1])

In [22]:
from collections import deque
x=deque(range(10),maxlen=10)

In [23]:
x

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [24]:
x[-1]

9

In [25]:
x= deque(range(5),maxlen=10)

In [26]:
x[-1]

4